# 버전 확인

In [1]:
import sys

sys.version

'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

In [2]:
import torch

torch.cuda.is_available()

True

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14390579297527567353
xla_global_id: -1
]


In [4]:
import tensorflow as tf

tf.__version__

'2.15.0'

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


In [6]:
!nvidia-smi

Fri Feb 16 19:00:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  | 00000000:08:00.0  On |                  N/A |
| 31%   46C    P0              28W / 125W |    710MiB /  6144MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1660 SUPER'

In [1]:
import glob
import os
from glob import glob
import yaml

os.environ["GIT_PYTHON_REFRESH"] = "quiet"

In [2]:
def translabel(foldername, after_label):
    # 폴더 안에 있는 txt 파일 리스트
    filelist = glob.glob('{}\\*.txt'.format(foldername))

    # 변환시킨 후 넣을 폴더 생성
    if not os.path.isdir('trans_'+foldername):
        os.mkdir('trans_'+foldername)

    # 변환
    for filename in filelist:
        with open(filename, 'r') as f:
            txtlines = ''
            lines = f.readlines()
            for line in lines:
                # 첫번째 공백 찾기 (기존 라벨 찾아내기1)
                first_blank = line.find(' ')

                # 공백앞의 문자
                pre_num = line[0:first_blank]

                # 공백앞의 문자 제거
                rm_pre_num = line.lstrip(pre_num)

                # 바꿀 문장
                after_line = str(after_label) + rm_pre_num
                txtlines += after_line

            # 바뀐것 생성
            with open('trans_' + filename, 'w') as f2:
                f2.write(txtlines)

# 이미지와 가중치 설정

In [3]:
img_list = glob('.\\image_data\\*.jpeg') # 학습시킬 이미지가 있는 파일 경로
img_list2 = glob('.\\image_data\\*.jpg') # 학습시킬 이미지가 있는 파일 경로

for i in img_list2:
  img_list.append(i)
print(img_list)

from sklearn.model_selection import train_test_split
# train test split
train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=129) # 8:2

# train test split하고 난 파일들의 경로를 저장하고 있는 txt 파일
with open('./image_data/data.yaml', 'r') as f:
  data = yaml.load(f, Loader=yaml.FullLoader)

with open('./yolov5/train.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(list(map(str, train_img_list))) + '\n')

with open('./yolov5/val.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(list(map(str, val_img_list))) + '\n')

with open('./image_data/data.yaml', 'w', encoding='utf-8') as f:
    yaml.dump(data, f, default_flow_style=False, allow_unicode=True)

print(data)

data['train'] = 'train.txt' # yolov5 경로
data['val'] = 'val.txt' # yolov5 경로

with open('./image_data/data.yaml', 'w') as f:
  yaml.dump(data, f)

['.\\image_data\\birdsongmush (1).jpeg', '.\\image_data\\birdsongmush (10).jpeg', '.\\image_data\\birdsongmush (11).jpeg', '.\\image_data\\birdsongmush (12).jpeg', '.\\image_data\\birdsongmush (13).jpeg', '.\\image_data\\birdsongmush (14).jpeg', '.\\image_data\\birdsongmush (15).jpeg', '.\\image_data\\birdsongmush (16).jpeg', '.\\image_data\\birdsongmush (17).jpeg', '.\\image_data\\birdsongmush (18).jpeg', '.\\image_data\\birdsongmush (19).jpeg', '.\\image_data\\birdsongmush (2).jpeg', '.\\image_data\\birdsongmush (20).jpeg', '.\\image_data\\birdsongmush (3).jpeg', '.\\image_data\\birdsongmush (4).jpeg', '.\\image_data\\birdsongmush (5).jpeg', '.\\image_data\\birdsongmush (6).jpeg', '.\\image_data\\birdsongmush (7).jpeg', '.\\image_data\\birdsongmush (8).jpeg', '.\\image_data\\birdsongmush (9).jpeg', '.\\image_data\\bread (1).jpeg', '.\\image_data\\bread (10).jpeg', '.\\image_data\\bread (11).jpeg', '.\\image_data\\bread (12).jpeg', '.\\image_data\\bread (13).jpeg', '.\\image_data\\bre

# 모델 학습

In [4]:
!python yolov5/train.py --img 416 --batch 16 --epochs 3 --data image_data/data.yaml --cfg yolov5/models/yolov5s.yaml --weights yolov5s.pt --name food_yolov5s_results --device cpu

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5



train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=image_data/data.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5\data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=cpu, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5\runs\train, name=food_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
YOLOv5  2024-1-18 Python-3.11.5 torch-2.1.1 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, a

# 모델 평가

In [5]:
!python yolov5/detect.py --weights yolov5/runs/train/food_yolov5s_results/weights/best.pt --img 416 --conf 0.2 --source 20240213_114040.jpg

detect: weights=['yolov5/runs/train/food_yolov5s_results/weights/best.pt'], source=20240213_114040.jpg, data=yolov5\data\coco128.yaml, imgsz=[416, 416], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2024-1-18 Python-3.11.5 torch-2.1.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7066762 parameters, 0 gradients, 15.9 GFLOPs
WARNING  NMS time limit 0.550s exceeded
image 1/1 C:\Users\Bro\Desktop\PlayData_Final-asdf\20240213_114040.jpg: 416x320 1 potato, 1 onion, 2 eggs, 1 enoki mushroom, 36.0ms
Speed: 1.0ms pre-process, 36.0ms inference, 911.0ms NMS per image at shape (1, 3, 416, 416)
Results saved to yolov5